for breif matrix explanations read-> https://github.com/USEPA/USEEIO_API/blob/master/doc/data_format.md

In [1]:
#BASE URL AND KEY
#Use the requests library
import csv
import zipfile, io
import os
import pathlib
import requests as r
import pandas as pd
pd.options.display.max_rows = 999

base_url ='https://smmtool.app.cloud.gov/api/' 
api_headers = {}
# api-key for USEEIO
with open(".USEEIO_API_KEY", 'r') as KEY_FILE:
    api_headers['x-api-key']=KEY_FILE.read()
#api_headers['x-api-key']="lySopVteG11Ru0m5ucnRharYBWco1CIGWlxKvro0"

In [7]:
# AVAILABLE MODELS
# Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()

# MODEL NAME
#Use the first available model
model_name = models_json[1]['id']

#MODEL SECTORS
#Get the sectors
model_sectors = r.get(base_url+model_name+'/sectors',headers=api_headers)
model_sectors_json = model_sectors.json()
print(model_sectors_json)
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')

# INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

[{'id': '1111a0/fresh soybeans, canola, flaxseeds, and other oilseeds/us-ga', 'index': 0, 'name': 'Fresh soybeans, canola, flaxseeds, and other oilseeds', 'code': '1111A0', 'location': 'US-GA', 'description': "BEA Code & Name is '1111A0:Oilseed farming'. This industry comprises establishments primarily engaged in growing oilseed crops. These crops have an annual life cycle and are typically grown in open fields. Related NAICS codes are: 11111-2"}, {'id': '1111b0/fresh wheat, corn, rice, and other grains/us-ga', 'index': 1, 'name': 'Fresh wheat, corn, rice, and other grains', 'code': '1111B0', 'location': 'US-GA', 'description': "BEA Code & Name is '1111B0:Grain farming'. This industry comprises establishments primarily engaged in producing grain seeds. These crops have an annual life cycle and are typically grown in open fields. Related NAICS codes are: 11113-6, 11119"}, {'id': '111200/fresh vegetables, melons, and potatoes/us-ga', 'index': 2, 'name': 'Fresh vegetables, melons, and pot

The GAUSEEIO model has 24 indicators. The names are:
['Acid Rain', 'Energy', 'Freshwater Aquatic Ecotoxicity', 'Eutrophication', 'Food Waste', 'Global Climate Change', 'Hazardous Air Pollutants', 'Hazardous Waste', 'Human Health Cancer', 'Human Health Noncancer', 'Human Health - Respiratory Effects', 'Human Health Cancer and Noncancer', 'Employment', 'Land', 'Metals', 'Minerals and Metals', 'Muncipal Solid Waste', 'Nonrenewable Energy', 'Ozone Depletion', 'Pesticides', 'Renewable Energy', 'Smog Formation', 'Value Added', 'Water']


In [62]:
model_indictrs_ids

['ACID',
 'ENRG',
 'ETOX',
 'EUTR',
 'FOOD',
 'GCC',
 'HAPS',
 'HAZW',
 'HC',
 'HNC',
 'HRSP',
 'HTOX',
 'JOBS',
 'LAND',
 'METL',
 'MINE',
 'MSW',
 'NREN',
 'OZON',
 'PEST',
 'REN',
 'SMOG',
 'VADD',
 'WATR']

In [3]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
model_indictrs_json

[{'id': 'ACID',
  'index': 0,
  'name': 'Acid Rain',
  'code': 'ACID',
  'unit': 'kg SO2 eq',
  'group': 'Impact Potential'},
 {'id': 'ENRG',
  'index': 1,
  'name': 'Energy',
  'code': 'ENRG',
  'unit': 'MJ',
  'group': 'Resource Use'},
 {'id': 'ETOX',
  'index': 2,
  'name': 'Freshwater Aquatic Ecotoxicity',
  'code': 'ETOX',
  'unit': 'CTUe',
  'group': 'Impact Potential'},
 {'id': 'EUTR',
  'index': 3,
  'name': 'Eutrophication',
  'code': 'EUTR',
  'unit': 'kg N eq',
  'group': 'Impact Potential'},
 {'id': 'FOOD',
  'index': 4,
  'name': 'Food Waste',
  'code': 'FOOD',
  'unit': 'kg',
  'group': 'Waste Generated'},
 {'id': 'GCC',
  'index': 5,
  'name': 'Global Climate Change',
  'code': 'GCC',
  'unit': 'kg CO2 eq',
  'group': 'Impact Potential'},
 {'id': 'HAPS',
  'index': 6,
  'name': 'Hazardous Air Pollutants',
  'code': 'HAPS',
  'unit': 'kg',
  'group': 'Chemical Releases'},
 {'id': 'HAZW',
  'index': 7,
  'name': 'Hazardous Waste',
  'code': 'HAZW',
  'unit': 'kg',
  'group

In [4]:
# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
# D0 = D0_response.json()
# sorted(D0)

D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

# sorted(D0[0])

In [65]:
#
# Collect ALL Indicators from ALL Industr Sectors and Limit to Georgia
#

# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

indicator_data = {}
# Add the industry sector IDs
# indicator_data.update({'industry':model_sector_ids})
indicator_data.update({'industry_code':[_.split('/')[0] for _ in model_sector_ids]})
indicator_data.update({'industry_detail':[_.split('/')[1] for _ in model_sector_ids]})
indicator_data.update({'industry_region':[_.split('/')[2] for _ in model_sector_ids]})


for key, values_list in zip(model_indictrs_ids, D0):
    indicator_data.update({key:values_list})

all_indic_all_indust = pd.DataFrame(data=indicator_data)
# all_indic_all_indust.loc[0:20,['industry_code', 'industry_detail', 'ACID']].sort_values(by='ACID', ascending=False)

all_indic_all_indust.head(50)

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us,0.003038,0.000000e+00,31.608836,0.011122,0.000026,0.558816,2.451175e-05,...,3.697394e-09,0.000000,0.000099,0.000000,0.000000e+00,3.661750e-04,0.000000e+00,0.022389,0.471064,0.002513
1,1111b0,"fresh wheat, corn, rice, and other grains",us,0.020069,0.000000e+00,12.615068,0.032458,0.000122,1.978520,6.667140e-05,...,1.154869e-06,0.000000,0.000472,0.000000,2.278307e-09,6.212053e-04,0.000000e+00,0.055404,-0.002548,0.460768
2,111200,"fresh vegetables, melons, and potatoes",us,0.007174,3.225479e-03,28.396543,0.006305,0.001194,0.457168,2.851897e-04,...,2.445284e-06,0.000000,0.004637,0.000000,7.327268e-06,1.075716e-03,3.225479e-03,0.021458,0.604597,0.591335
3,111300,fresh fruits and tree nuts,us,0.003656,0.000000e+00,20.718815,0.000275,0.001945,0.286005,3.849088e-05,...,1.547424e-05,0.000000,0.007557,0.000000,1.289248e-06,4.691275e-04,0.000000e+00,0.018263,0.605345,0.456172
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us,0.000952,5.123345e-06,0.000677,0.000118,0.001786,0.709750,4.436491e-05,...,2.509153e-08,0.000000,0.006971,0.000000,8.190052e-10,3.562148e-14,5.123345e-06,0.037800,0.534020,0.046371
5,111900,"tobacco, cotton, sugarcane, peanuts, sugar bee...",us,0.005891,5.868687e-03,23.786667,0.005384,0.000512,1.083857,2.140583e-04,...,7.328570e-09,0.000000,0.001989,0.000000,2.602376e-09,3.243746e-04,5.868687e-03,0.045542,0.466042,3.553048
6,112120,dairies,us,0.019774,0.000000e+00,0.000321,0.001723,0.000595,2.530155,2.500158e-05,...,4.197728e-09,0.000000,0.002311,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.026072,0.325316,0.058702
7,1121a0,cattle ranches and feedlots,us,0.016596,0.000000e+00,0.000412,0.001592,0.000164,2.911778,3.207974e-05,...,5.631707e-09,0.000000,0.000636,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.032946,0.295837,0.083773
8,112300,poultry farms,us,0.024104,0.000000e+00,0.000098,0.001558,0.000215,0.308309,7.594978e-06,...,1.519250e-09,0.000000,0.000837,0.000000,8.669449e-14,0.000000e+00,0.000000e+00,0.007409,0.231893,0.008426
9,112a00,animal farms and aquaculture ponds (except cat...,us,0.031055,1.752462e-05,0.000139,0.002216,0.000486,1.084541,1.092244e-05,...,1.839515e-08,0.000000,0.001892,0.000000,7.040062e-12,3.191010e-12,1.752462e-05,0.011054,0.624700,0.450648


In [66]:
all_indic_all_indust.min()

industry_code                   1111a0
industry_detail      abrasive products
industry_region    gas field machinery
ACID                                 0
ENRG                                 0
ETOX                                 0
EUTR                      -2.55231e-08
FOOD                                 0
GCC                                  0
HAPS                                 0
HAZW                                 0
HC                                   0
HNC                                  0
HRSP                                 0
HTOX                                 0
JOBS                                 0
LAND                                 0
METL                      -1.08526e-10
MINE                                 0
MSW                                  0
NREN                                 0
OZON                                 0
PEST                      -2.27252e-14
REN                                  0
SMOG                                 0
VADD                     

In [67]:
all_indic_all_indust=all_indic_all_indust[all_indic_all_indust['ENRG'] !=0]

In [68]:
all_indic_all_indust.min()

industry_code                 111200
industry_detail    abrasive products
industry_region                   us
ACID                     7.11381e-08
ENRG                     4.09471e-09
ETOX                     2.00838e-08
EUTR                    -2.55231e-08
FOOD                               0
GCC                      0.000212623
HAPS                     1.02198e-08
HAZW                               0
HC                       1.10792e-14
HNC                      2.15618e-14
HRSP                     9.28547e-09
HTOX                      3.2663e-14
JOBS                      5.0741e-09
LAND                     1.77919e-05
METL                    -1.08526e-10
MINE                               0
MSW                                0
NREN                               0
OZON                               0
PEST                    -2.27252e-14
REN                                0
SMOG                     2.99038e-06
VADD                       0.0617671
WATR                     1.38524e-05
d

In [69]:
all_indic_all_indust=all_indic_all_indust.replace(0,0.0000000000000000000000001)

In [70]:
all_indic_all_indust.to_csv('indicators_sectors.csv')